In [24]:
%%javascript 
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [25]:
%autoreload 2
%matplotlib inline 
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10, 8)

In [74]:
!du -h ../../data/*05_13_16.h5

 22M	../../data/test_stars_fhat_clst5_05_13_16.h5
 84K	../../data/test_stars_peak_df_clst5_05_13_16.h5


In [75]:
from __future__ import print_function
import pandas as pd
import h5py
import sys
sys.path.append("../")
import extract_catalog as ec
import get_DM_centroids as getDM
import get_gal_centroids as getgal
import seaborn as sns

In [76]:
import compute_distance as compDist
import plot_DM_prop as plotDM

In [77]:
sns.set_style('white')

In [78]:
data_path = "../../data/"

In [79]:
clstNo = 5
date_stamp = '05_13_16'
DM_df = \
    pd.read_hdf(data_path + 
                "test_DM_peaks_df_clst{0}_{1}.h5".format(clstNo, 
                                                         date_stamp), 
                "peak_df")
DM_fhat = \
    h5py.File(data_path + 
              "test_DM_fhat_clst{0}_{1}.h5".format(clstNo, 
                                                   date_stamp))

IOError: File ../../data/test_DM_peaks_df_clst5_05_13_16.h5 does not exist

In [80]:
star_fhat_file = "test_stars_fhat_clst{0}_{1}.h5".format(
    clstNo, date_stamp)
star_fhat = \
    h5py.File(data_path + 
              "test_stars_fhat_clst{0}_{1}.h5".format(clstNo, 
                                                      date_stamp))

star_df = \
    pd.read_hdf(data_path + 
                "test_stars_peak_df_clst{0}_{1}.h5".format(
            clstNo, date_stamp),
                "peak_df")

In [82]:
star_fhat_path = []
star_fhat.visit(star_fhat_path.append)

In [89]:
star_fhat['/4']

<HDF5 group "/4" (1 members)>

In [83]:
star_fhat_path[-1]

u'4/min/i_band/1/1.5707963268/5.8904862255'

In [81]:
star_df

,peaks_xcoords,peaks_ycoords,peaks_dens,clstNo,cut,weights,los_axis,xi,phi
0,-43.560884,-14.852330,1.000000,0,min,i_band,1,0.4111378623,0.7853981634
1,-6.177046,-1617.616122,0.202993,0,min,i_band,1,0.4111378623,0.7853981634
2,1040.570426,-261.431375,0.047161,0,min,i_band,1,0.4111378623,0.7853981634
3,-1351.995224,-2521.739287,0.040267,0,min,i_band,1,0.4111378623,0.7853981634
4,-1538.914415,-1987.484690,0.026691,0,min,i_band,1,0.4111378623,0.7853981634
5,-305.247752,1094.753372,0.017070,0,min,i_band,1,0.4111378623,0.7853981634
0,-16.894551,-20.263867,1.000000,0,min,i_band,1,0.4111378623,2.3561944902
1,922.388651,-1994.859653,0.162088,0,min,i_band,1,0.4111378623,2.3561944902
2,209.828980,1449.202764,0.050029,0,min,i_band,1,0.4111378623,2.3561944902
3,-1247.679437,-2545.909640,0.044036,0,min,i_band,1,0.4111378623,2.3561944902


# Visualize the smoothed and not smoothed histograms

In [45]:
gal_metadata = getgal.metadata_from_h5_key_path(path)

In [46]:
gal_metadata

OrderedDict([('phi', u'0.7853981634'),
             ('xi', u'0.4111378623'),
             ('cut', u'min'),
             ('weights', u'i_band'),
             ('clstNo', u'13'),
             ('los_axis', u'1')])

In [33]:
DM_gpBy_key_no = 8 
star_gpBy_key_no = len(star_df.keys()) - 3
DM_path_list = \
    compDist.retrieve_cluster_path(DM_fhat, DM_gpBy_key_no)

In [49]:
DM_path_list[0].split('/')

[u'13',
 u'min',
 u'i_band',
 u'1',
 u'0.4111378623',
 u'0.7853981634',
 u'0.20',
 u'0']

In [34]:
DM_gp_keys = tuple(DM_df.keys()[-9:])
gal_gp_keys = tuple(star_df.keys()[-6:])


In [41]:
DM_df.keys()

Index([u'peaks_xcoords', u'peaks_ycoords', u'peaks_dens', u'clstNo', u'cut', u'weights', u'los_axis', u'xi', u'phi', u'sig_fraction', u'kernel_width'], dtype='object')

In [35]:
DM_gp_keys

('peaks_dens',
 'clstNo',
 'cut',
 'weights',
 'los_axis',
 'xi',
 'phi',
 'sig_fraction',
 'kernel_width')

In [36]:
gal_gp_keys

('clstNo', 'cut', 'weights', 'los_axis', 'xi', 'phi')

In [37]:
DM_gpBy = DM_df.groupby(DM_gp_keys, as_index=False)

In [38]:
len(DM_gp_keys)

9

In [40]:
path

u'13/min/i_band/1/0.4111378623/0.7853981634/0.20/0'

In [ ]:
compDist.DM_h5path_to_groupbykey(path)

In [73]:
DM_fhat.visit(print)

13
13/min
13/min/i_band
13/min/i_band/1
13/min/i_band/1/0.4111378623
13/min/i_band/1/0.4111378623/0.0000000000
13/min/i_band/1/0.4111378623/0.0000000000/0.0
13/min/i_band/1/0.4111378623/0.0000000000/25.0
13/min/i_band/1/0.4111378623/0.3926990817
13/min/i_band/1/0.4111378623/0.3926990817/0.0
13/min/i_band/1/0.4111378623/0.3926990817/25.0
13/min/i_band/1/0.4111378623/0.7853981634
13/min/i_band/1/0.4111378623/0.7853981634/0.0
13/min/i_band/1/0.4111378623/0.7853981634/0.20
13/min/i_band/1/0.4111378623/0.7853981634/0.20/0
13/min/i_band/1/0.4111378623/0.7853981634/0.20/0/estimate
13/min/i_band/1/0.4111378623/0.7853981634/0.20/0/eval_points0
13/min/i_band/1/0.4111378623/0.7853981634/0.20/0/eval_points1
13/min/i_band/1/0.4111378623/0.7853981634/0.20/25
13/min/i_band/1/0.4111378623/0.7853981634/0.20/25/estimate
13/min/i_band/1/0.4111378623/0.7853981634/0.20/25/eval_points0
13/min/i_band/1/0.4111378623/0.7853981634/0.20/25/eval_points1
13/min/i_band/1/0.4111378623/0.7853981634/25.0
13/min/i_band

In [17]:
for path in DM_path_list:
    clstNo = path.split('/')[0]
    DM_clst_dict = \
        compDist.combine_DM_df_and_h5_to_dict(
            DM_gpBy, DM_fhat, path)
    star_dict = compDist.get_star_dict_from_run(
        star_df, star_fhat, path, star_gpBy_key_no)
    plotDM.plot_DM_fhat(
        fhat=DM_clst_dict, fhat_stars=star_dict, clstNo=clstNo)
    plt.show()
    plt.close()

ValueError: must supply a a same-length tuple to get_group with multiple grouping keys

> /Users/karenyng/Documents/illustris_analyses/code/IlEnv/lib/python2.7/site-packages/pandas/core/groupby.py(439)_get_index()
    437                     msg = ("must supply a a same-length tuple to get_group"
    438                            " with multiple grouping keys")
--> 439                     raise ValueError(msg)
    440 
    441             name = tuple([ convert(n, k) for n, k in zip(name,sample) ])

ipdb> up
> /Users/karenyng/Documents/illustris_analyses/code/IlEnv/lib/python2.7/site-packages/pandas/core/groupby.py(597)get_group()
    595             obj = self._selected_obj
    596 
--> 597         inds = self._get_index(name)
    598         return obj.take(inds, axis=self.axis, convert=False)
    599 

ipdb> up
> /Users/karenyng/Documents/illustris_analyses/code/compute_distance.py(165)combine_DM_df_and_h5_to_dict()
    163     gpbykeys = DM_h5path_to_groupbykey(h5path)
    164     # May want to convert some of the quantities back to float!?
--> 165     fhat =

In [18]:
DM_gp_keys

('peaks_dens',
 'clstNo',
 'cut',
 'weights',
 'los_axis',
 'xi',
 'phi',
 'sig_fraction',
 'kernel_width')

In [19]:
DM_df.keys()[-9:]

Index([u'peaks_dens', u'clstNo', u'cut', u'weights', u'los_axis', u'xi', u'phi', u'sig_fraction', u'kernel_width'], dtype='object')